TensorFlow-多层感知机

In [1]:
#导入NumPy
import numpy as np
#导入TensorFlow
import tensorflow as tf

In [2]:
'''
在TensorFlow中，图像数据集的一种典型表示是[图像数目,长,宽,色彩通道数]的四维张量。
由于这里读入的是灰度图片，色彩通道数为1（彩色 RGB 图像色彩通道数为3），所以我们使用np.expand_dims()函数为图像数据手动在最后添加一维通道。
'''
#图像特征抽取
class MNISTLoader():
    def __init__(self):
        #获取MNIST手写数字数据集
        mnist=tf.keras.datasets.mnist
        #划分数据集
        (self.train_data,self.train_label),(self.test_data,self.test_label)=mnist.load_data()
        #抽取图像特征值
        '''
        MNIST中的图像默认为uint8（0-255的数字），以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道；
        np.expand_dims()用来扩展数组的维度，axis=-1表示数组的最后一个维度。
        '''
        self.train_data=np.expand_dims(self.train_data.astype(np.float32)/255.0,axis=-1)      
        self.test_data=np.expand_dims(self.test_data.astype(np.float32)/255.0,axis=-1)
        #获取目标值
        self.train_label=self.train_label.astype(np.int32) 
        self.test_label=self.test_label.astype(np.int32)
        #样本量
        self.num_train_data,self.num_test_data=self.train_data.shape[0],self.test_data.shape[0]

    #从数据集中随机取出batch_size个元素并返回
    def get_batch(self,batch_size):
        #np.random.randint()在[0,self.num_train_data)范围内抽取batch_size个整数作为索引
        index=np.random.randint(0,self.num_train_data,batch_size)
        #返回索引对应的样本
        return self.train_data[index,:],self.train_label[index]

In [3]:
#多层感知机
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        #Flatten层将除第一维（图像数目batch_size）以外的维度展平
        self.flatten=tf.keras.layers.Flatten()
        #隐含层（节点数为100，以ReLU函数激活）
        self.dense1=tf.keras.layers.Dense(units=100,activation=tf.nn.relu)
        #输出层（节点数为10，对应数字0~9）
        self.dense2=tf.keras.layers.Dense(units=10)

    #输入张量的形状为[batch_size,28,28,1]
    def call(self,inputs):
        #展平后的张量形状为[batch_size,784]
        x=self.flatten(inputs)
        #隐含层输出的张量形状为[batch_size,100]
        x=self.dense1(x)    
        #输出层输出的张量形状为[batch_size,10]
        x=self.dense2(x)
        #以Softmax函数激活分类结果
        output=tf.nn.softmax(x)          
        return output

In [4]:
#设置超参数
#遍历神经网络的次数
num_epochs=5
#每次迭代采用的样本量
batch_size=50
#学习率
learning_rate=0.001

In [5]:
#调用模型
model=MLP()
#调取数据
data_loader=MNISTLoader()
#调用优化器
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [6]:
#迭代次数（//表示整除）
num_batches=int(num_epochs*data_loader.num_train_data//batch_size)
#梯度下降
for batch_index in range(num_batches):
    #训练集的特征值和目标值
    X,y=data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        #预测值
        y_pred=model(X)
        #交叉熵损失函数
        #计算交叉熵
        loss=tf.keras.losses.sparse_categorical_crossentropy(y_true=y,y_pred=y_pred)
        #计算不同维度上交叉熵的平均值
        loss=tf.reduce_mean(loss)
        print("batch %d: loss %f"%(batch_index,loss.numpy()))
    #计算损失函数对变量（参数和偏置）的梯度
    grads=tape.gradient(loss,model.variables)
    #将梯度和变量（参数和偏置）输入优化器
    optimizer.apply_gradients(grads_and_vars=zip(grads,model.variables))

batch 0: loss 2.390351
batch 1: loss 2.234385
batch 2: loss 2.199473
batch 3: loss 2.103645
batch 4: loss 2.148223
batch 5: loss 2.017295
batch 6: loss 2.015996
batch 7: loss 1.860679
batch 8: loss 1.986476
batch 9: loss 1.851929
batch 10: loss 1.696756
batch 11: loss 1.738537
batch 12: loss 1.620730
batch 13: loss 1.576076
batch 14: loss 1.456794
batch 15: loss 1.250115
batch 16: loss 1.446651
batch 17: loss 1.392096
batch 18: loss 1.371196
batch 19: loss 1.276368
batch 20: loss 1.403111
batch 21: loss 1.252634
batch 22: loss 1.100818
batch 23: loss 1.254942
batch 24: loss 1.162307
batch 25: loss 0.975452
batch 26: loss 1.095115
batch 27: loss 1.004615
batch 28: loss 0.866720
batch 29: loss 1.100735
batch 30: loss 0.943835
batch 31: loss 0.863378
batch 32: loss 1.000891
batch 33: loss 0.836718
batch 34: loss 0.747438
batch 35: loss 0.792416
batch 36: loss 0.775457
batch 37: loss 0.719979
batch 38: loss 0.735951
batch 39: loss 0.977959
batch 40: loss 0.586628
batch 41: loss 0.658322
ba

batch 353: loss 0.370250
batch 354: loss 0.213322
batch 355: loss 0.134745
batch 356: loss 0.323674
batch 357: loss 0.252248
batch 358: loss 0.250890
batch 359: loss 0.189000
batch 360: loss 0.253099
batch 361: loss 0.254785
batch 362: loss 0.413076
batch 363: loss 0.327749
batch 364: loss 0.450562
batch 365: loss 0.368387
batch 366: loss 0.202281
batch 367: loss 0.243412
batch 368: loss 0.241223
batch 369: loss 0.196766
batch 370: loss 0.256132
batch 371: loss 0.100154
batch 372: loss 0.110427
batch 373: loss 0.241044
batch 374: loss 0.394121
batch 375: loss 0.277597
batch 376: loss 0.252624
batch 377: loss 0.445811
batch 378: loss 0.207417
batch 379: loss 0.210674
batch 380: loss 0.292785
batch 381: loss 0.329953
batch 382: loss 0.386225
batch 383: loss 0.263767
batch 384: loss 0.229847
batch 385: loss 0.228037
batch 386: loss 0.492297
batch 387: loss 0.425377
batch 388: loss 0.326666
batch 389: loss 0.186284
batch 390: loss 0.313593
batch 391: loss 0.186823
batch 392: loss 0.390866


batch 683: loss 0.135081
batch 684: loss 0.301831
batch 685: loss 0.240982
batch 686: loss 0.165840
batch 687: loss 0.078025
batch 688: loss 0.237416
batch 689: loss 0.234902
batch 690: loss 0.357904
batch 691: loss 0.345052
batch 692: loss 0.124540
batch 693: loss 0.145781
batch 694: loss 0.313577
batch 695: loss 0.331018
batch 696: loss 0.421557
batch 697: loss 0.167663
batch 698: loss 0.288110
batch 699: loss 0.098544
batch 700: loss 0.197931
batch 701: loss 0.142657
batch 702: loss 0.322466
batch 703: loss 0.260682
batch 704: loss 0.131771
batch 705: loss 0.134252
batch 706: loss 0.143395
batch 707: loss 0.228953
batch 708: loss 0.187812
batch 709: loss 0.224303
batch 710: loss 0.147905
batch 711: loss 0.092727
batch 712: loss 0.313916
batch 713: loss 0.061667
batch 714: loss 0.082758
batch 715: loss 0.238157
batch 716: loss 0.062479
batch 717: loss 0.515929
batch 718: loss 0.200807
batch 719: loss 0.118902
batch 720: loss 0.143624
batch 721: loss 0.186419
batch 722: loss 0.134050


batch 1049: loss 0.229424
batch 1050: loss 0.217234
batch 1051: loss 0.173895
batch 1052: loss 0.216839
batch 1053: loss 0.130259
batch 1054: loss 0.255519
batch 1055: loss 0.183244
batch 1056: loss 0.154348
batch 1057: loss 0.086464
batch 1058: loss 0.471525
batch 1059: loss 0.150669
batch 1060: loss 0.129131
batch 1061: loss 0.165052
batch 1062: loss 0.099203
batch 1063: loss 0.082623
batch 1064: loss 0.107705
batch 1065: loss 0.156408
batch 1066: loss 0.201639
batch 1067: loss 0.280804
batch 1068: loss 0.136842
batch 1069: loss 0.409025
batch 1070: loss 0.451146
batch 1071: loss 0.116487
batch 1072: loss 0.217082
batch 1073: loss 0.449337
batch 1074: loss 0.082369
batch 1075: loss 0.134977
batch 1076: loss 0.069156
batch 1077: loss 0.089891
batch 1078: loss 0.066355
batch 1079: loss 0.240410
batch 1080: loss 0.226373
batch 1081: loss 0.063442
batch 1082: loss 0.234655
batch 1083: loss 0.190992
batch 1084: loss 0.104224
batch 1085: loss 0.168061
batch 1086: loss 0.087497
batch 1087: 

batch 1392: loss 0.155898
batch 1393: loss 0.083039
batch 1394: loss 0.156276
batch 1395: loss 0.169672
batch 1396: loss 0.098227
batch 1397: loss 0.164349
batch 1398: loss 0.225296
batch 1399: loss 0.122801
batch 1400: loss 0.042022
batch 1401: loss 0.074263
batch 1402: loss 0.082852
batch 1403: loss 0.067860
batch 1404: loss 0.061955
batch 1405: loss 0.090846
batch 1406: loss 0.073104
batch 1407: loss 0.040798
batch 1408: loss 0.090751
batch 1409: loss 0.119488
batch 1410: loss 0.208676
batch 1411: loss 0.121679
batch 1412: loss 0.088457
batch 1413: loss 0.093915
batch 1414: loss 0.220469
batch 1415: loss 0.123894
batch 1416: loss 0.109687
batch 1417: loss 0.217142
batch 1418: loss 0.108848
batch 1419: loss 0.232077
batch 1420: loss 0.195980
batch 1421: loss 0.139434
batch 1422: loss 0.075951
batch 1423: loss 0.249245
batch 1424: loss 0.074543
batch 1425: loss 0.162678
batch 1426: loss 0.323258
batch 1427: loss 0.067598
batch 1428: loss 0.125052
batch 1429: loss 0.144157
batch 1430: 

...

batch 4334: loss 0.020064
batch 4335: loss 0.030025
batch 4336: loss 0.072880
batch 4337: loss 0.033614
batch 4338: loss 0.035814
batch 4339: loss 0.081409
batch 4340: loss 0.087156
batch 4341: loss 0.027076
batch 4342: loss 0.142099
batch 4343: loss 0.118871
batch 4344: loss 0.078594
batch 4345: loss 0.021501
batch 4346: loss 0.165043
batch 4347: loss 0.210289
batch 4348: loss 0.066086
batch 4349: loss 0.013897
batch 4350: loss 0.177572
batch 4351: loss 0.130307
batch 4352: loss 0.034256
batch 4353: loss 0.083321
batch 4354: loss 0.222829
batch 4355: loss 0.073643
batch 4356: loss 0.082945
batch 4357: loss 0.034366
batch 4358: loss 0.027253
batch 4359: loss 0.126484
batch 4360: loss 0.014129
batch 4361: loss 0.049988
batch 4362: loss 0.022384
batch 4363: loss 0.024318
batch 4364: loss 0.062607
batch 4365: loss 0.021183
batch 4366: loss 0.106266
batch 4367: loss 0.170553
batch 4368: loss 0.037658
batch 4369: loss 0.249093
batch 4370: loss 0.023540
batch 4371: loss 0.039124
batch 4372: 

batch 4662: loss 0.016424
batch 4663: loss 0.039698
batch 4664: loss 0.021132
batch 4665: loss 0.010788
batch 4666: loss 0.013477
batch 4667: loss 0.091812
batch 4668: loss 0.029182
batch 4669: loss 0.248611
batch 4670: loss 0.129861
batch 4671: loss 0.022567
batch 4672: loss 0.061196
batch 4673: loss 0.125054
batch 4674: loss 0.014960
batch 4675: loss 0.084508
batch 4676: loss 0.006759
batch 4677: loss 0.043246
batch 4678: loss 0.035001
batch 4679: loss 0.175372
batch 4680: loss 0.085079
batch 4681: loss 0.027255
batch 4682: loss 0.090753
batch 4683: loss 0.036751
batch 4684: loss 0.129468
batch 4685: loss 0.059701
batch 4686: loss 0.039649
batch 4687: loss 0.036056
batch 4688: loss 0.016648
batch 4689: loss 0.013161
batch 4690: loss 0.040003
batch 4691: loss 0.152040
batch 4692: loss 0.031567
batch 4693: loss 0.048140
batch 4694: loss 0.016157
batch 4695: loss 0.083468
batch 4696: loss 0.022912
batch 4697: loss 0.215661
batch 4698: loss 0.012512
batch 4699: loss 0.100304
batch 4700: 

batch 4993: loss 0.091173
batch 4994: loss 0.094186
batch 4995: loss 0.024996
batch 4996: loss 0.048945
batch 4997: loss 0.036709
batch 4998: loss 0.031250
batch 4999: loss 0.114524
batch 5000: loss 0.125190
batch 5001: loss 0.052363
batch 5002: loss 0.008985
batch 5003: loss 0.020478
batch 5004: loss 0.024237
batch 5005: loss 0.034545
batch 5006: loss 0.054685
batch 5007: loss 0.037732
batch 5008: loss 0.073739
batch 5009: loss 0.062384
batch 5010: loss 0.025025
batch 5011: loss 0.202211
batch 5012: loss 0.094192
batch 5013: loss 0.069010
batch 5014: loss 0.059017
batch 5015: loss 0.073853
batch 5016: loss 0.028442
batch 5017: loss 0.029505
batch 5018: loss 0.118844
batch 5019: loss 0.026686
batch 5020: loss 0.039773
batch 5021: loss 0.085544
batch 5022: loss 0.193712
batch 5023: loss 0.066422
batch 5024: loss 0.020925
batch 5025: loss 0.083881
batch 5026: loss 0.034508
batch 5027: loss 0.034128
batch 5028: loss 0.063110
batch 5029: loss 0.024573
batch 5030: loss 0.081606
batch 5031: 

batch 5317: loss 0.018734
batch 5318: loss 0.041969
batch 5319: loss 0.040552
batch 5320: loss 0.144603
batch 5321: loss 0.044787
batch 5322: loss 0.056225
batch 5323: loss 0.011677
batch 5324: loss 0.161196
batch 5325: loss 0.092076
batch 5326: loss 0.028865
batch 5327: loss 0.115765
batch 5328: loss 0.042429
batch 5329: loss 0.020670
batch 5330: loss 0.044438
batch 5331: loss 0.039889
batch 5332: loss 0.080367
batch 5333: loss 0.047405
batch 5334: loss 0.016838
batch 5335: loss 0.038837
batch 5336: loss 0.038187
batch 5337: loss 0.044400
batch 5338: loss 0.058379
batch 5339: loss 0.022747
batch 5340: loss 0.227350
batch 5341: loss 0.009939
batch 5342: loss 0.026587
batch 5343: loss 0.066879
batch 5344: loss 0.040723
batch 5345: loss 0.079252
batch 5346: loss 0.017011
batch 5347: loss 0.230629
batch 5348: loss 0.098734
batch 5349: loss 0.019472
batch 5350: loss 0.124077
batch 5351: loss 0.048668
batch 5352: loss 0.034788
batch 5353: loss 0.072643
batch 5354: loss 0.040324
batch 5355: 

batch 5662: loss 0.113925
batch 5663: loss 0.016253
batch 5664: loss 0.045271
batch 5665: loss 0.013975
batch 5666: loss 0.114727
batch 5667: loss 0.143088
batch 5668: loss 0.049945
batch 5669: loss 0.123202
batch 5670: loss 0.007297
batch 5671: loss 0.037397
batch 5672: loss 0.034533
batch 5673: loss 0.055699
batch 5674: loss 0.005025
batch 5675: loss 0.077429
batch 5676: loss 0.063133
batch 5677: loss 0.084540
batch 5678: loss 0.079333
batch 5679: loss 0.046428
batch 5680: loss 0.045272
batch 5681: loss 0.046105
batch 5682: loss 0.077431
batch 5683: loss 0.092391
batch 5684: loss 0.050150
batch 5685: loss 0.040682
batch 5686: loss 0.054266
batch 5687: loss 0.013757
batch 5688: loss 0.025988
batch 5689: loss 0.049902
batch 5690: loss 0.054785
batch 5691: loss 0.037379
batch 5692: loss 0.011152
batch 5693: loss 0.048995
batch 5694: loss 0.062630
batch 5695: loss 0.094763
batch 5696: loss 0.060364
batch 5697: loss 0.029727
batch 5698: loss 0.103917
batch 5699: loss 0.016670
batch 5700: 

In [7]:
#调用评估器
sparse_categorical_accuracy=tf.keras.metrics.SparseCategoricalAccuracy()
#迭代次数
num_batches=int(data_loader.num_test_data//batch_size)
for batch_index in range(num_batches):
    #设置索引
    start_index,end_index=batch_index*batch_size,(batch_index+1)*batch_size
    #预测值
    y_pred=model.predict(data_loader.test_data[start_index:end_index])
    #计算精确率
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index],y_pred=y_pred)
print("test accuracy: %f"%sparse_categorical_accuracy.result())

test accuracy: 0.973100
